Genetic algorithms were developed by John Holland in 1992 as a method to select the best hyperparameter values for computational algorithms. There are often so many hyperparameters that affect an algorithms success, and each hyperparameter has many possible values. This results in an insurmountably large search space of potential hyperparameters that would take way too long for a poor data scientist to explore manually. Instead, genetic algorithms start with a population of random hyperparameter combinations and lets them compete in a process resembling natural selection. The combination (or chromosome) of hyperparameter values that creates the most successful algorithm emerges as the most fit. This process involves mating, crossover, and mutation, similar to natural selection in the wild. The code below is adapted from https://machinelearningmastery.com/simple-genetic-algorithm-from-scratch-in-python/.

In [59]:
import pandas as pd
import numpy as np
from numpy.random import randint
from numpy.random import rand

### Create a population of random bitstrings
First we generate a starting population. Each "individual" is conceptualized as a chromosome of bitstrings. Each value on the chromosome represents a hyperparameter value. “n_pop” is a hyperparameter that controls the population size and “n_bits” is a hyperparameter that defines the number of bits in a single candidate solution.

In [133]:
# Initial population size: 
n_pop = 100

# Number of hyperparameters per chromosome: 10
n_bits = 20

pop = [np.random.randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
pop

[[1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1],
 [1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0],
 [1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
 [1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1],
 [0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1],
 [1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0],
 [1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0],
 [1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1],
 [1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0],
 [1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0],
 [1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0],
 [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

### Define how you will evaluate the chromosome's fitness
In this case, I will simply sum up the number of ones in the chromosome. A higher sum will be considered more fit. Because the code eventually will be written so that lower scores have better fitness values, I have to add a negative number so that higher sums have better fitness scores. The optimal fitness score will be -20 because there are 20 bits in the chromosome.

In [134]:
def onemax(x):
    return -sum(x)

### Define the number of generations (iterations)
Each iteration allows the chromosomes to compete in a new generation

In [135]:
# Number of generations (iterations) desired
n_iter = 100

### Define the crossover rate and define crossover function
Crossovers mimic genetic recombination. A crossover point in the "chromosome" is determined where the chromosome from one parent is joined to the chromosome of the other parent. Here we use a crossover rate of 90%.

In [159]:
r_cross = 0.9

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < r_cross:
        # select crossover point that is not on the end of the string
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

### Define the mutation rate and define mutation operator
It is a good idea for the mutation rate to be 1/L, where L is the length of the chromosome. Here, the mutation rate will be 5%.

In [164]:
r_mut = 1.0 / float(n_bits)

# mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]

### Select parents from each generation using tournament selection
Parents are selected based on their fitness. A given candidate solution may be used as parent zero or more times. A simple and effective approach to selection involves drawing k candidates from the population randomly and selecting the member from the group with the best fitness. This is called tournament selection where k is a hyperparameter and set to a value such as 3. This simple approach simulates a more costly fitness-proportionate selection scheme.

In [165]:
# tournament selection
def selection(pop, scores, k=3):
    # first random selection
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k-1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

### Define the genetic algorithm

In [168]:
def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut):

    # initial population of random bitstring
    pop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)]

    # keep track of best solution
    best, best_eval = 0, objective(pop[0])

    # enumerate generations
    for gen in range(n_iter):
        # evaluate all candidates in the population
        scores = [objective(c) for c in pop]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %.3f" % (gen,  pop[i], scores[i]))

        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]

        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]

            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)

                # store for next generation
                children.append(c)

        # replace population
        pop = children

    return [best, best_eval]

### Perform the genetic algorithm search that maximizes the number of 1s on the chromosome

In [169]:
best, score = genetic_algorithm(onemax, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
print('f(%s) = %f' % (best, score))

>0, new best f([0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1]) = -10.000
>0, new best f([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1]) = -12.000
>0, new best f([0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1]) = -13.000
>0, new best f([1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1]) = -14.000
>0, new best f([1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1]) = -15.000
>1, new best f([1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -18.000
>7, new best f([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -19.000
>8, new best f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -20.000
Done!
f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -20.000000
